In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cadd_io as cdio
import cadd_main as cdmain
import mdutilities_io as mduio
import mymath as Mmath
import my_plot as myplot
import cadddatadump as cddump
import cadd_mesh2 as cdmesh
import cadd
import itertools
import newpotential as newpot
import tabularpotential as tabpot
import crystallography_oo as croo
import lineplot_oo3 as lpoo
%matplotlib qt
%load_ext autoreload
%autoreload 2

# Preliminaries

## Directories

In [2]:
maindir = '../Tests/CADD_NoDisl_K_Test/'
uidir = maindir + 'User Inputs/'
fidir = maindir + 'Fortran Inputs/'
dumpdir = maindir + 'Dump Files/'
simname = 'cadd_nodisl_k_test'
simtype = 'cadd_nodisl'

In [3]:
size = 'large'
if size == 'small':
    ferings = 5
elif size == 'medium':
    ferings = 14
elif size == 'large':
    ferings = 24
simname += '_' + size

## Unit System

In [4]:
# LJ?

# Create Inputs

In [5]:
r0 = 1.0

## Nodes/Elements

### Create Mesh

In [6]:
padx, pady = 5, 5
lx, ly = 42, 45 # gives roughly 2500 atoms (including pad, interface)
xy = cadd.simple_hex_lattice(lx+padx,ly+pady,r0=r0) # make box slightly larger to accommodate pad
meshnodes = cdmesh.Nodes(xy)
mesh = cdmesh.MeshTri(nodes=meshnodes)
meshnodes = mesh.nodes
boxold = meshnodes.gen_interface_and_pad(np.array([-lx/2,lx/2,-ly/2,ly/2]))
for i in range(ferings):
    nnodesvec = [line.get_nnodes() for line in boxold.lines]
    if i < 10:
        if i%3 == 0:
            ring = cdmesh.RingCollapseMixed(boxold)
        else:
            ring = cdmesh.RingExtend(boxold)
    else:
        ring = cdmesh.RingCollapseMixed(boxold)
    mesh.add_ring(ring)
    boxold = ring.boxnew

In [7]:
# k-field will be imposed on outer boundary
outernodes = boxold.all_nodes
meshnodes.set_node_bc(outernodes,3) # outer nodes are completely fixed

### Fix Mesh to Account for Crack Plane

In [8]:
# crack line and nodes
idx = np.argmin(np.abs(meshnodes.xy[:,1]))
ycrack = meshnodes.xy[idx,1]
crackline = cdmesh.Line([-np.inf,ycrack],[0.0,ycrack])
nodenums = meshnodes.search_for_nodes_along_line(crackline)

In [9]:
# construct new nodes
# the bit with ydisp is a bit kludgy, and might depend on the specific mesh...
mapping = {}
ydisp = r0*np.sqrt(3)/2 # atomic spacing in y-direction
ydispvec = np.array([0,ydisp])
ydispvecatom = np.array([r0/2,ydisp])
defaultfetype = cdmesh.Nodes.defaultfetype
for nodenum in nodenums:
    posn = meshnodes.xy[nodenum,:]
    nodetype = meshnodes.types[nodenum,:]
    if nodetype[1] in [0,2]: # FE node or interface atom
        if nodetype[1] == 2: # interface atom must be moved
            posnnew = posn + ydispvecatom
            nodenew = meshnodes.closest_node(posnnew)
        else:
            posnnew = posn + ydispvec
            nodenew = meshnodes.add_node(posnnew,defaultfetype)
        meshnodes.types[nodenew,:] = nodetype # retain type and bc of old node
        mapping[nodenum] = nodenew  

In [10]:
# this modifies the element while it's being looped over, which is dangerous
for i, element in enumerate(mesh.elements):
    common = np.intersect1d(element,nodenums,assume_unique=True)
    if common.size: # nodenum belongs to element
        pts = meshnodes.get_points(element)
        yelementcenter = np.mean(pts,axis=0)[1]
        if Mmath.same_sign(yelementcenter-ycrack,ydisp): # only create a new element if the element center is on one side of ycrack
            for nodeold in common:
                nodenew = mapping[nodeold]
                np.place(element,element==nodeold,nodenew) # replace old num with new num

In [11]:
# fudge fe node positions slightly to ensure pad atoms are within an element
yfudge = ydisp/20
ydispvec = np.array([0,np.copysign(yfudge,ydisp)])
for nodeold, nodenew in mapping.items():
    if meshnodes.types[nodeold,1] == 0: # not an interface node (which should remain fixed)
        meshnodes.xy[nodeold,:] += ydispvec
        meshnodes.xy[nodenew,:] -= ydispvec

### Plot

In [12]:
mesh.write_dump_all('test.6.dump')
sim = cdmain.Simulation('cadd_nodisl','test',readinput=False)
fig = sim.plot_dump_from_file('test.6.dump')

### Nodes

In [13]:
n = meshnodes.xy.shape[0]
z = np.zeros((n,))
posn = np.column_stack((meshnodes.xy,z)) # z-coordinate = 0
nodes = cdmain.Nodes(posn=posn,types=meshnodes.types)

### FE Elements

In [14]:
feelements = cdmain.FEElement(elname='CPE3',mnum=1,connect=mesh.elements_dump) # elements_dump fixes off by 1 indexing issue

## Potential

In [15]:
brittlenew = newpot.gen_std_potential(ductilityindex=0,dimensions=2,r0=1)

In [16]:
rminsmall, rmin, rmax = 0.01, 0.1, 2.0
npoints = 3000
rvec = np.linspace(rmin,rmax,npoints)
rvec = np.insert(rvec,0,rminsmall)
pottable = brittlenew.get_energy_force_table(rvec)

In [17]:
brittlenew.write_file_lammps('brittlenewnorm',np.linspace(0.1,3.0,3000),extend=True)

In [18]:
# Energy plot
# lpoo.my_quick_plot([pottable[:,[0,1]]],axisbounds=[0.5,2,-1,3])

In [19]:
rnnn = np.sqrt(3)*r0 # 2nd nearest neighbor
potential = cdmain.Potential(pname='brittlenew',forcecutoff=rnnn,pottable=pottable)

## Material

### Elastic constants

In [20]:
elasticdict = brittlenew.elasticdict('hex')
brittlemat = croo.Hex.from_potential(brittlenew)
elconst = brittlemat.voigt_plane_strain_stiffness

In [21]:
elconst

array([[ 40.09879362,  13.36626454,   0.        ],
       [ 13.36626454,  40.09879362,   0.        ],
       [  0.        ,   0.        ,  13.36626454]])

In [22]:
material = cdmain.Material(burgers=r0,disldrag=0.0,dislvmax=0.0,elconst=elconst,lannih=0.0,lattice='hex',mass=1.0,
                           mname='material1',rho=brittlenew.rho)

## Misc

In [23]:
misc = cdmain.Misc(increments=100,timestep=0.02,iscrackproblem=True,potstyle='table')

## Interactions

In [24]:
interactions = cdmain.Interactions(table=np.array([[1,1,1]]))

## Neighbors

In [25]:
neighbors = cdmain.Neighbors(checkdisp=True,every=0,delay=5,images=0,Lz=1.0,skin=0.3)

## Damping

In [26]:
gamma = 0.1 # may need to fiddle with this
damping = cdmain.Damping(flag=True,gname='all',gamma=gamma)

## Everything

In [27]:
caddsim = cdmain.Simulation(simtype,simname,uidir,fidir,dumpdir,readinput=False)
caddsim.data = cdmain.CADDData(simtype,nfematerials=1,nodes=nodes,materials=[material],misc=misc,groups=None,
                               potentials=[potential],interactions=interactions,neighbors=neighbors,damping=damping,
                               feelements=[feelements])


In [28]:
caddsim.data.check_data()

## Write

In [29]:
caddsim.write_fortran_all()

# Misc

# Dump Files

In [39]:
fig = caddsim.plot_dump_from_increment(7,tight=False,fignum=1,axisbounds=[-30,30,-30,30]);

In [40]:
fig = caddsim.plot_dump_from_increment(9,tight=False,fignum=2,axisbounds=[-30,30,-30,30]);

In [32]:
brittlenew.rho

1.1547005383792517

# Log Files

## Determine K

In [33]:
brittlekmat = croo.Hex.from_potential(brittlenew)

In [34]:
aold = np.eye(3).astype(int)
anew = np.array([[-1,1,0],[1,1,-2],[1,1,1]])
burgers = np.array([1,0,-1])

In [35]:
Kicgriffith, Kierice = brittlekmat.k_both(aold,anew,burgers,printoption=True)

Crack Plane: [ 1  1 -2]
Slip Plane: [-1  2 -1]
Theta: 59.99999999999999 (degrees)
Phi: 1.2074182697257333e-06 (degrees)
K_ic: 8.443147767084215
K_ie: 13.422972533947563


## Measured KIc

In [41]:
Kic = 9.2

## Comparison

In [42]:
Gratio = (Kic/Kicgriffith)**2
print(Gratio)

1.18731751506


In [38]:
# in paper (Rajan, Warner, Curtin, 2016, MSMSE):
# GIc (MS)/GIc (Griffith) = 1.18